# Laboratory excercise 1 and 2
The notebook contains excercise connected to the auditory excerices 1 and 2. For any questions feel free to contact assistans: eda.jovicic@fer.hr and filip.tomas@fer.hr

In [2]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.model_selection import cross_val_score
from sklearn.feature_selection import SelectKBest, mutual_info_classif

In [3]:
import warnings
warnings.filterwarnings('ignore')

1. Load the dataset *student_score.csv*. When loading the dataset watch out for the index column. 

In [4]:
file_path = r"C:\Users\ONE\OneDrive - FFUNIZG\Radna površina\Faks\CogSci\DataScience\data\students_score.csv"
student_data = pd.read_csv(file_path, index_col=0)

2. Print the size of the dataset. List the number of examples and the number of features. 

In [5]:
student_data.shape

(30641, 15)

In [11]:
st_data_dict={"Number of examples": student_data.shape[0], "Number of features": student_data.shape[1]}
st_data_list = student_data.shape

3. Show the first 5 examples of the dataset. 

In [9]:
student_data.head(5)

,StudentId,Gender,EthnicGroup,ParentEduc,LunchType,TestPrep,ParentMaritalStatus,PracticeSport,IsFirstChild,NrSiblings,TransportMeans,WklyStudyHours,MathScore,ReadingScore,WritingScore
0,520645,female,NaN,bachelor's degree,standard,none,married,regularly,yes,3.0,school_bus,< 5,71,71,74
1,303683,female,group C,some college,standard,NaN,married,sometimes,yes,0.0,NaN,05-Oct,69,90,88
2,457351,female,group B,master's degree,standard,none,single,sometimes,yes,4.0,school_bus,< 5,87,93,91
3,812988,male,group A,associate's degree,free/reduced,none,married,never,no,1.0,NaN,05-Oct,45,56,42
4,594678,male,group C,some college,standard,none,married,sometimes,yes,0.0,school_bus,05-Oct,76,78,75


4. Using the third example from the dataset print for every feature the type of data (int, float, string or bool).

In [9]:
data_types = student_data.iloc[2].map(type)
print(data_types)

StudentId                <class 'numpy.int64'>
Gender                           <class 'str'>
EthnicGroup                      <class 'str'>
ParentEduc                       <class 'str'>
LunchType                        <class 'str'>
TestPrep                         <class 'str'>
ParentMaritalStatus              <class 'str'>
PracticeSport                    <class 'str'>
IsFirstChild                     <class 'str'>
NrSiblings             <class 'numpy.float64'>
TransportMeans                   <class 'str'>
WklyStudyHours                   <class 'str'>
MathScore                <class 'numpy.int64'>
ReadingScore             <class 'numpy.int64'>
WritingScore             <class 'numpy.int64'>
Name: 2, dtype: object


5. For numerical features print the value of mean, standard deviation and median. 


In [14]:
num_features = ["StudentId", "NrSiblings", "MathScore", "ReadingScore", "WritingScore"]
descriptives_dict = {"mean": student_data[num_features].mean(),
    "standard deviation": student_data[num_features].std(),
    "median": student_data[num_features].median()}
statistics = pd.DataFrame(descriptives_dict)
statistics

,mean,standard deviation,median
StudentId,549614.852257,257913.251180,550904.0
NrSiblings,2.147305,1.466746,2.0
MathScore,66.558402,15.361616,67.0
ReadingScore,69.377533,14.758952,70.0
WritingScore,68.418622,15.443525,69.0


6. Check if there are any monotonous attributes. If you find a monotonous attribute, remove it from the dataset.

In [15]:
student_data.nunique()

StudentId              30641
Gender                     2
EthnicGroup                5
ParentEduc                 6
LunchType                  2
TestPrep                   2
ParentMaritalStatus        4
PracticeSport              3
IsFirstChild              12
NrSiblings                11
TransportMeans             2
WklyStudyHours             3
MathScore                 95
ReadingScore              90
WritingScore              93
dtype: int64

In [16]:
student_data.drop('StudentId', axis=1, inplace=True)

In [17]:
student_data.columns

Index(['Gender', 'EthnicGroup', 'ParentEduc', 'LunchType', 'TestPrep',
       'ParentMaritalStatus', 'PracticeSport', 'IsFirstChild', 'NrSiblings',
       'TransportMeans', 'WklyStudyHours', 'MathScore', 'ReadingScore',
       'WritingScore'],
      dtype='object')

7. Check the data for missing values. Print the number of missing values for each feature.

In [18]:
student_data.isna().sum()

Gender                    0
EthnicGroup            1840
ParentEduc             1845
LunchType                 0
TestPrep               1830
ParentMaritalStatus    1190
PracticeSport           631
IsFirstChild            903
NrSiblings             1572
TransportMeans         3134
WklyStudyHours          955
MathScore                 0
ReadingScore              0
WritingScore              0
dtype: int64

8. Let's deal with the missing values. 

a) For the features EthnicGroup, ParentMaritalStatus and WklyStudyHours replace NaN values with the most common value.

b) For the feature ParentEduc replace NaN value with a string *no formal education*.

c) For the feature TestPrep replace NaN value with a string *none*.

d) For the features PracticeSport and IsFirstChild remove all the examples with NaN values. 

e) For the feature NrSiblings replace the NaN value with the median value.

f) Finally, for the feature TransportMeans replace the NaN value with a string *walking*.




In [19]:
student_data_tmp = student_data.copy()

In [20]:
feature_group1 = ["EthnicGroup", "ParentMaritalStatus", "WklyStudyHours"]
for feature in feature_group1:
    student_data_tmp.loc[student_data_tmp[feature].isna(), feature] = student_data_tmp.loc[:,feature].value_counts().idxmax()

In [21]:
student_data_tmp.loc[student_data_tmp["ParentEduc"].isnull(), "ParentEduc"] = "no formal education"
student_data_tmp.loc[student_data_tmp["TestPrep"].isnull(), "TestPrep"] = "none"

student_data_tmp = student_data_tmp.loc[student_data_tmp["PracticeSport"].notnull(),:]
student_data_tmp = student_data_tmp.loc[student_data_tmp["IsFirstChild"].notnull(),:]

student_data_tmp.loc[student_data_tmp.NrSiblings.isna(), "NrSiblings"] = student_data_tmp.NrSiblings.median() 
    #or should it be the median of the population prior to removing those with null in PracticeSport and IsFirstChild?
    #if so, the preceding line of code should simply have been put and executed before the two lines with "PracticeSport" 
    #and "IsFirstChild" where parts of the data are eliminated.

student_data_tmp.loc[student_data_tmp.TransportMeans.isnull(), "TransportMeans"] = "walking"

In [22]:
student_data_tmp.isna().sum()

Gender                 0
EthnicGroup            0
ParentEduc             0
LunchType              0
TestPrep               0
ParentMaritalStatus    0
PracticeSport          0
IsFirstChild           0
NrSiblings             0
TransportMeans         0
WklyStudyHours         0
MathScore              0
ReadingScore           0
WritingScore           0
dtype: int64

9. Are there inconsistencies in the IsFirstChild column? Replace the inconsistent entries so they match the rest of the values. 

In [23]:
student_data_tmp["IsFirstChild"].unique()

array(['yes', 'no', 'nein', 'nope', 'ye', 'si', 'Yes', 'yeah', 'noo',
       'ys', 'yeah boy', 'es'], dtype=object)

In [24]:
student_data_tmp["IsFirstChild"].value_counts()

IsFirstChild
yes         18669
no          10405
noo             7
nope            6
ye              6
Yes             6
nein            5
yeah            4
ys              3
si              2
yeah boy        2
es              2
Name: count, dtype: int64

In [25]:
student_data_tmp["IsFirstChild"] = student_data_tmp["IsFirstChild"].replace(["ye", "si", "Yes", "yeah", "ys", "yeah boy", "es"], "yes")
student_data_tmp["IsFirstChild"] = student_data_tmp["IsFirstChild"].replace(["nein", "nope", "noo"], "no")

In [26]:
student_data_tmp

,Gender,EthnicGroup,ParentEduc,LunchType,TestPrep,ParentMaritalStatus,PracticeSport,IsFirstChild,NrSiblings,TransportMeans,WklyStudyHours,MathScore,ReadingScore,WritingScore
0,female,group C,bachelor's degree,standard,none,married,regularly,yes,3.0,school_bus,< 5,71,71,74
1,female,group C,some college,standard,none,married,sometimes,yes,0.0,walking,05-Oct,69,90,88
2,female,group B,master's degree,standard,none,single,sometimes,yes,4.0,school_bus,< 5,87,93,91
3,male,group A,associate's degree,free/reduced,none,married,never,no,1.0,walking,05-Oct,45,56,42
4,male,group C,some college,standard,none,married,sometimes,yes,0.0,school_bus,05-Oct,76,78,75
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
816,female,group D,high school,standard,none,single,sometimes,no,2.0,school_bus,05-Oct,59,61,65
890,male,group E,high school,standard,none,single,regularly,no,1.0,private,05-Oct,58,53,51
911,female,group C,high school,free/reduced,completed,married,sometimes,no,1.0,private,05-Oct,61,70,67
934,female,group D,associate's degree,standard,completed,married,regularly,no,3.0,school_bus,05-Oct,82,90,93


10. The final step to preparing the dataset for classification is transforming the non-numeric variables to numeric variables. 
For features Gender and LunchType use OneHot Encoder, and for EthnicGroup, ParentEduc, TestPrep, ParentMaritalStatus, PracticeSport, IsFirstChild, TransportMeans and WklyStudyHours use Label Encoder. 

In [27]:
encoder = OneHotEncoder(handle_unknown = "ignore")
   
encoder.fit(student_data_tmp[["Gender","LunchType"]])
student_encoder_tmp = encoder.transform(student_data_tmp[["Gender","LunchType"]])
student_encoder_tmp = pd.DataFrame(student_encoder_tmp.toarray(), columns=encoder.get_feature_names_out(["Gender","LunchType"]))

student_encoder_tmp

,Gender_female,Gender_male,LunchType_free/reduced,LunchType_standard
0,1.0,0.0,0.0,1.0
1,1.0,0.0,0.0,1.0
2,1.0,0.0,0.0,1.0
3,0.0,1.0,1.0,0.0
4,0.0,1.0,0.0,1.0
...,...,...,...,...
29112,1.0,0.0,0.0,1.0
29113,0.0,1.0,0.0,1.0
29114,1.0,0.0,1.0,0.0
29115,1.0,0.0,0.0,1.0


In [28]:
student_data_tmp = student_data_tmp.join(student_encoder_tmp)
student_data_tmp

,Gender,EthnicGroup,ParentEduc,LunchType,TestPrep,ParentMaritalStatus,PracticeSport,IsFirstChild,NrSiblings,TransportMeans,WklyStudyHours,MathScore,ReadingScore,WritingScore,Gender_female,Gender_male,LunchType_free/reduced,LunchType_standard
0,female,group C,bachelor's degree,standard,none,married,regularly,yes,3.0,school_bus,< 5,71,71,74,1.0,0.0,0.0,1.0
0,female,group C,some college,standard,none,divorced,never,yes,0.0,private,05-Oct,72,86,82,1.0,0.0,0.0,1.0
0,female,group E,associate's degree,free/reduced,none,single,regularly,yes,0.0,school_bus,< 5,74,77,68,1.0,0.0,0.0,1.0
0,male,group B,bachelor's degree,free/reduced,none,married,sometimes,yes,4.0,private,> 10,62,52,53,1.0,0.0,0.0,1.0
0,female,group C,associate's degree,free/reduced,none,married,sometimes,no,2.0,private,05-Oct,79,89,82,1.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999,male,group C,high school,standard,none,single,sometimes,no,1.0,private,> 10,79,74,69,1.0,0.0,0.0,1.0
999,male,group C,high school,standard,none,married,regularly,yes,2.0,school_bus,< 5,66,63,57,1.0,0.0,0.0,1.0
999,female,group C,high school,free/reduced,completed,married,sometimes,yes,3.0,school_bus,> 10,38,57,59,1.0,0.0,0.0,1.0
999,female,group D,associate's degree,standard,none,married,regularly,yes,1.0,private,05-Oct,53,69,67,1.0,0.0,0.0,1.0


In [29]:
le = LabelEncoder()
le_features = ["EthnicGroup", "ParentEduc", "TestPrep", "ParentMaritalStatus", "PracticeSport", "IsFirstChild", "TransportMeans", "WklyStudyHours"]
for feature in le_features:
    student_data_tmp[feature] = le.fit_transform(student_data_tmp[feature])

In [28]:
student_data_tmp.head()

,Gender,EthnicGroup,ParentEduc,LunchType,TestPrep,ParentMaritalStatus,PracticeSport,IsFirstChild,NrSiblings,TransportMeans,WklyStudyHours,MathScore,ReadingScore,WritingScore,Gender_female,Gender_male,LunchType_free/reduced,LunchType_standard
0,female,2,1,standard,1,1,1,1,3.0,1,1,71,71,74,1.0,0.0,0.0,1.0
0,female,2,5,standard,1,0,0,1,0.0,0,0,72,86,82,1.0,0.0,0.0,1.0
0,female,4,0,free/reduced,1,2,1,1,0.0,1,1,74,77,68,1.0,0.0,0.0,1.0
0,male,1,1,free/reduced,1,1,2,1,4.0,0,2,62,52,53,1.0,0.0,0.0,1.0
0,female,2,0,free/reduced,1,1,2,0,2.0,0,0,79,89,82,1.0,0.0,0.0,1.0


In [32]:
student_data = student_data_tmp.copy()

In [34]:
student_data.reset_index(drop=True, inplace=True)

In [35]:
student_data

,Gender,EthnicGroup,ParentEduc,LunchType,TestPrep,ParentMaritalStatus,PracticeSport,IsFirstChild,NrSiblings,TransportMeans,WklyStudyHours,MathScore,ReadingScore,WritingScore,Gender_female,Gender_male,LunchType_free/reduced,LunchType_standard
0,female,2,1,standard,1,1,1,1,3.0,1,1,71,71,74,1.0,0.0,0.0,1.0
1,female,2,5,standard,1,0,0,1,0.0,0,0,72,86,82,1.0,0.0,0.0,1.0
2,female,4,0,free/reduced,1,2,1,1,0.0,1,1,74,77,68,1.0,0.0,0.0,1.0
3,male,1,1,free/reduced,1,1,2,1,4.0,0,2,62,52,53,1.0,0.0,0.0,1.0
4,female,2,0,free/reduced,1,1,2,0,2.0,0,0,79,89,82,1.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29112,male,2,2,standard,1,2,2,0,1.0,0,2,79,74,69,1.0,0.0,0.0,1.0
29113,male,2,2,standard,1,1,1,1,2.0,1,1,66,63,57,1.0,0.0,0.0,1.0
29114,female,2,2,free/reduced,0,1,2,1,3.0,1,2,38,57,59,1.0,0.0,0.0,1.0
29115,female,3,0,standard,1,1,1,1,1.0,0,0,53,69,67,1.0,0.0,0.0,1.0


In [53]:
file_path = r"C:\Users\ONE\OneDrive - FFUNIZG\Radna površina\Faks\CogSci\DataScience\data\student_data1.csv"
student_data.to_csv(file_path, index=False)